# Elasto-plastic isotropic hardening model

Verification of the Legendre transform

In [ ]:
%matplotlib widget
from bmcs_matmod import GSMRM
from bmcs_matmod import GSMNR
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from bmcs_utils.api import Cymbol, cymbols
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_d_lih import Potential1D_T_E_VP_D_LIH_SymbExpr 
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_lih import Potential1D_T_E_VP_LIH_SymbExpr 
sp.init_printing()

## Direct solution of consistency condition

### Helmholtz free energy

In [ ]:
p1d = Potential1D_T_E_VP_D_LIH_SymbExpr()
p1d.F_

### GSM drivers

In [ ]:
gsm_F = GSMRM(
    name = 'gsm_F_1d_t_e_vp_d_lkh',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

gsm_L = GSMNR(
    name = 'gsm_F_1d_t_e_vp_d_lkh_l',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

In [ ]:
gsm_F.Sig_.as_explicit(), gsm_L.Sig_.as_explicit()

In [ ]:
R_, _, _, _, _ = gsm_L.f_R_dR_n1
R_

In [ ]:
from matplotlib.pylab import f
mparams_epd = dict(E_=1, K_lin_=1e-5, f_c_=1, S_=0.1, c_=0.1, r_=1, eta_=1, C_v_=1, T_0_=20)
# eps_n_ = np.array([1.0, -1.0, 0, 0], dtype=np.float_)
# d_eps_ = np.array([0.1, -0.1, 2, -2], dtype=np.float_)
eps_n_ = np.array([0], dtype=np.float_)
d_eps_ = np.array([2], dtype=np.float_)
d_A_ = np.zeros((eps_n_.shape[0], gsm_L.n_Eps_explicit+1), dtype=np.float_)
Eps_n_ = np.zeros((eps_n_.shape[0], gsm_L.n_Eps_explicit), dtype=np.float_)
tol = 1e-8
max_iter = 150
for k in range(max_iter):
    f_n1_, R_n1_, dR_n1_ = gsm_L.get_f_R_dR_n1(eps_n_, d_eps_, Eps_n_, d_A_, 1, **mparams_epd)
    norm_R_n1_ = np.linalg.norm(R_n1_, axis=-1)
    I_ = norm_R_n1_ >= tol
    if np.all(I_ == False):
        print(f'|R| = {norm_R_n1_}, f = {f_n1_}, k = {k}')
        break
    d_A_[I_] += np.linalg.solve(dR_n1_[I_], -R_n1_[I_])
    d_A_[d_A_[..., 2] > 1] = 0.9999
print('d_A_', d_A_)

In [ ]:
mparams_epd = dict(E_=1, K_lin_=1e-5, f_c_=1, S_=0.1, c_=0.1, r_=1, eta_=1, C_v_=1, T_0_=20)
eps_n_I = np.array([0, 1.0, 1.0, -1.0, 0, 0], dtype=np.float_)
d_eps_I = np.array([0.1, -0.1, 0.1, -0.1, 2, -2], dtype=np.float_)
n_I = eps_n_I.shape[0]
d_t = 1
Eps_n_Ib = np.zeros((n_I, gsm_L.n_Eps_explicit), dtype=np.float_)

Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I = gsm_L.get_state_n1(eps_n_I, d_eps_I, d_t, Eps_n_Ib, 30, **mparams_epd )
Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I

In [ ]:
mparams_epd = dict(E_=1, K_lin_=10, f_c_=1, S_=0.3, c_=0.1, r_=1, eta_=1, C_v_=1, T_0_=20)

n_t = 51
n_I = 1
eps_max = 6
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_L.n_Eps_explicit), dtype=np.float_)
t_t, eps_ta_nr_coarse, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_L.get_response(eps_n_t, t_t, 100, **mparams_epd)

In [ ]:
n_t = 2000
n_I = 1
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_L.n_Eps_explicit), dtype=np.float_)
t_t, eps_ta_nr_fine, Eps_t, Sig_t_nr_fine, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_L.get_response(eps_n_t, t_t, 30, **mparams_epd)

In [ ]:
n_t = 51
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)[:, np.newaxis]
Eps_n_Ib = np.zeros((n_I, gsm_L.n_Eps_explicit), dtype=np.float_)
T_t = np.zeros_like(t_t) + 20
t_t, eps_ta_rm, T_t, Eps_t, Sig_t_rm, iter_t, dDiss_dEps_t, lam_t, (d_t_t, d_eps_ta) = gsm_F.get_response(eps_n_t, T_t, t_t, k_max=20, **mparams_epd)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(eps_ta_rm[:, 0, 0], Sig_t_rm[:,0,0], color='blue')
ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='red')
ax.plot(eps_ta_nr_fine, Sig_t_nr_fine[:,:,0], color='green')
